In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from utils import *
from pipelines import *
from sklearn.ensemble import RandomForestRegressor

# Read the data
X = pd.read_csv('./input/train.csv', index_col='Id') 
X_test = pd.read_csv('./input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

# X_test.dropna(axis=0, subset=['SalePrice'], inplace=True)
# y_test = X_test.SalePrice
# X_test.drop(['SalePrice'], axis=1, inplace=True)


In [ ]:
# X_fixed, X_fixed_test = fix_non_numerical_cols(X, X_test)
# X_fixed, X_fixed_test = fix_na_columns(X_fixed, X_fixed_test)

In [ ]:
# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [12]:
model = RandomForestRegressor(n_estimators=300, max_depth=100, criterion='absolute_error', random_state=0)
categorical_cols, numerical_cols, ordinal_cols = get_cols_by_type(X)
pipeline = get_default_pipeline(categorical_cols, numerical_cols, ordinal_cols)(model)
my_cols = categorical_cols + numerical_cols + ordinal_cols
X_trimmed = X[my_cols].copy()
X_train = X_train[my_cols].copy()
X_valid = X_valid[my_cols].copy()
X_test = X_test[my_cols].copy()


In [6]:
# pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'BsmtFullBath',
                                                   'BsmtHalfBath', 'FullBa...
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir',
                                                   'Electrical', 'KitchenQual',
                                                   'Functional', 'FireplaceQu', ...]),
                                                 ('ord',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OrdinalEncoder())]),
                                                  ['Neighborhood',
                                                   'Exterior1st',
                                                   'Exterior2nd'])])),
                ('model',
                 RandomForestRegressor(criterion='absolute_error',
                                       max_depth=100, n_estimators=300,
                                       random_state=0))])

In [7]:

# preds = pipeline.predict(X_valid)

# print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 17643.761255707763


In [13]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(pipeline, X_trimmed, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

C:\Users\Juan\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Juan\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Juan\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_scorer.py", line 106, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "C:\Users\Juan\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_scorer.py", line 261, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "C:\Users\Juan\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_scorer.py", line 71, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "C:\Users\Juan\AppData\Roam

In [ ]:
scores